In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

df = pd.read_csv("./new_preprocessed_final.csv")

In [2]:
# BERT similarity feature
bert_feat = None
for c in df.columns:
  cl = c.lower()
  if "bert_similarity" in cl or ("cosine" in cl and "bert" in cl):
    bert_feat = c
    break
if bert_feat is None:
  raise ValueError("BERT similarity column not found.")

# Extra features
extra_feats = ["title_length","sensational_word_count","punctuation_count","punctuation_ratio"]
extra_feats = [c for c in extra_feats if c in df.columns]

# Final features: BERT + extras
feature_cols = [bert_feat] + extra_feats

# Label
label_col = None
for c in df.columns:
  if "is_clickbait" in c.lower():
    label_col = c
    break

X = df[feature_cols]
y = df[label_col]

# Split
X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.2, random_state=42, stratify=y
)

# LR pipeline
pipe = Pipeline([
  ("scaler", StandardScaler()),
  ("clf", LogisticRegression(max_iter=300, class_weight="balanced"))
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

print("== Features used ==")
print(feature_cols)

print("\n=== Classification Report (LR + BERT + Extras) ===")
print(classification_report(y_test, y_pred))

print("\n=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred))

== Features used ==
['bert_similarity', 'title_length', 'sensational_word_count', 'punctuation_count', 'punctuation_ratio']

=== Classification Report (LR + BERT + Extras) ===
              precision    recall  f1-score   support

           0       0.90      0.59      0.71       514
           1       0.20      0.59      0.29        86

    accuracy                           0.59       600
   macro avg       0.55      0.59      0.50       600
weighted avg       0.80      0.59      0.65       600


=== Confusion Matrix ===
[[305 209]
 [ 35  51]]
